spotrac base 建模資料前處理

In [1]:
import sys
sys.path.append('G:\我的雲端硬碟\python')

from polib.CsvEngn import *
import pandas as pd
import numpy as np

# 讀取

In [2]:
FA_path = r"G:\我的雲端硬碟\python\app\MLB\spotrac_data\MLB Free Agents by Players  Spotrac.csv"
EX_path = r"G:\我的雲端硬碟\python\output\0825\extension_df_20210825_21.43.10.csv"
WAR_path = r"G:\我的雲端硬碟\python\output\0819\WAR_df_20210819_10.24.49.csv"
CC_path = r"G:\我的雲端硬碟\python\output\0826\CurentContract_df_20210826_12.57.41.csv"

FA_df = read_data_dir_fPathExt(FA_path)
EX_df = read_data_dir_fPathExt(EX_path)
WAR_df = read_data_dir_fPathExt(WAR_path)
CC_df = read_data_dir_fPathExt(CC_path)

  >> 讀取:: G:\我的雲端硬碟\python\app\MLB\spotrac_data\MLB Free Agents by Players  Spotrac.csv
  >> 讀取:: G:\我的雲端硬碟\python\output\0825\extension_df_20210825_21.43.10.csv
  >> 讀取:: G:\我的雲端硬碟\python\output\0819\WAR_df_20210819_10.24.49.csv
  >> 讀取:: G:\我的雲端硬碟\python\output\0826\CurentContract_df_20210826_12.57.41.csv


In [3]:
FA_df.head()

,Rank,Player,Year,Pos,Status,Team From,Team From To,YRS,Value,AAV,Unnamed: 10
0,NaN,Bryce Harper,2019,RF,UFA,WSH,PHI,13,"$330,000,000","$25,384,615",NaN
1,NaN,Gerrit Cole,2020,SP,UFA,HOU,NYY,9,"$324,000,000","$36,000,000",NaN
2,NaN,Manny Machado,2019,3B,UFA,LAD,SD,10,"$300,000,000","$30,000,000",NaN
3,NaN,Stephen Strasburg,2020,SP,UFA,WSH,WSH,7,"$245,000,000","$35,000,000",NaN
4,NaN,Anthony Rendon,2020,3B,UFA,WSH,LAA,7,"$245,000,000","$35,000,000",NaN


In [4]:
EX_df.head()

,_idx,Player,Pos.,Age @ Sign,Team @ Sign,Yrs,Dollars,AAV,2-Year Cash,3-Year Cash,year
0,0,Ryan Braun,1B,27,MIL,5,"$105,000,000","$21,000,000","$12,000,000","$21,500,000",2011
1,1,Koji Uehara,RP,35,BAL,1,"$3,000,000","$3,000,000",NaN,NaN,2011
2,2,Brandon Phillips,2B,30,CIN,6,"$72,500,000","$12,083,333","$22,575,000","$33,650,000",2012
3,3,Alexei Ramirez,SS,30,CHW,4,"$32,500,000","$8,125,000","$12,000,000","$21,515,000",2012
4,4,Clay Buchholz,SP,27,BOS,4,"$29,945,000","$7,486,250","$10,050,000","$17,750,000",2012


In [5]:
# 縮小 WAR_df的欄位範圍: 球季、球隊、球員、WAR
print(WAR_df.columns)
WAR_cols_pic = ["Season","player","Team","WAR"]
WAR_df = WAR_df[WAR_cols_pic]
WAR_df.head()

Index(['_idx', 'Season', 'Team', 'GP', 'GS', 'AB', 'R', 'H', '2B', '3B', 'HR',
       'RBI', 'BB', 'SB', 'K', 'AVG', 'OBP', 'SLG', 'OPS', 'WAR', 'player',
       'IP', 'ER', 'W', 'L', 'S', 'ERA', 'WHIP', 'Games', 'Date', 'Reason',
       'Days Missed', 'Cash Paid While Injured', 'Terms', 'Avg. Salary'],
      dtype='object')


,Season,player,Team,WAR
0,2016,Kelly Johnson,ATL,-0.73
1,2015,Kelly Johnson,ATL,-0.04
2,2014,Kelly Johnson,NYY,-0.08
3,2013,Kelly Johnson,TB,1.33
4,2012,Kelly Johnson,TOR,1.81


In [6]:
WAR_df.isna().sum()

Season     183
player       0
Team      1052
WAR        183
dtype: int64

In [7]:
# 刪除有缺值的資料
print(len(WAR_df))
WAR_df = WAR_df.dropna()
print(len(WAR_df))

8464
7409


# 變數製作

## 是否轉隊 

In [8]:
# Season排序是由新到舊
WAR_df['preSeasonTeam'] = WAR_df.groupby(["player"])['Team'].shift(-1)
WAR_df["is_changeTeam"] = False
WAR_df["is_changeTeam"] = (WAR_df["preSeasonTeam"].notna()) & (WAR_df['preSeasonTeam'] != WAR_df['Team'])
WAR_df.head()

,Season,player,Team,WAR,preSeasonTeam,is_changeTeam
0,2016,Kelly Johnson,ATL,-0.73,ATL,False
1,2015,Kelly Johnson,ATL,-0.04,NYY,True
2,2014,Kelly Johnson,NYY,-0.08,TB,True
3,2013,Kelly Johnson,TB,1.33,TOR,True
4,2012,Kelly Johnson,TOR,1.81,ARI,True


## 年齡、年齡平方 (串 CurentContract_df)

In [9]:
CC_df.Year.astype(int,errors="ignore")[0]

'2016'

In [10]:
# 以 CC_df 年齡紀錄反推選手生日年
# CC_df.columns
CC_df.loc[:,"Year"] = pd.to_numeric(CC_df.Year, errors='coerce').fillna(0).astype(np.int64)
CC_df.loc[:,"Age"] = pd.to_numeric(CC_df.Age, errors='coerce').fillna(0).astype(np.int64)
CC_df_f = CC_df[(CC_df["Year"]>0)&(CC_df["Age"]>0)]
CC_df_f = CC_df_f[["player", "Year","Age"]].groupby(["player"]).head(1)
CC_df_f.loc[:,"birth_year"] = CC_df_f.loc[:,"Year"] - CC_df_f.loc[:,"Age"] +1 
CC_df_f

,player,Year,Age,birth_year
0,Kelly Johnson,2016,34,1983
4,Joe Saunders,2015,33,1983
7,Chris Young,2018,39,1980
10,Stephen Drew,2017,34,1984
14,Justin Upton,2018,30,1989
...,...,...,...,...
4068,Hisashi Iwakuma,2018,37,1982
4072,Mike Zunino,2021,30,1992
4077,Marco Gonzales,2020,28,1993
4086,Joe Gatto,2021,26,1996


In [11]:
# 串生日，計算每年的Age
WAR_df = WAR_df.merge(CC_df_f[["player","birth_year"]], how="left", on="player")
WAR_df.loc[:,"birth_year"] = WAR_df.loc[:,"birth_year"].fillna(0).astype(np.int64)
WAR_df.loc[:,"Season"] = pd.to_numeric(WAR_df.loc[:,"Season"], errors='coerce').fillna(0).astype(np.int64)
WAR_df.loc[:,"Age"] = WAR_df.loc[:,"Season"] - WAR_df.loc[:,"birth_year"] +1
WAR_df.loc[:,"Age^2"] = WAR_df.loc[:,"Age"]**2
WAR_df

,Season,player,Team,WAR,preSeasonTeam,is_changeTeam,birth_year,Age,Age^2
0,2016,Kelly Johnson,ATL,-0.73,ATL,False,1983,34,1156
1,2015,Kelly Johnson,ATL,-0.04,NYY,True,1983,33,1089
2,2014,Kelly Johnson,NYY,-0.08,TB,True,1983,32,1024
3,2013,Kelly Johnson,TB,1.33,TOR,True,1983,31,961
4,2012,Kelly Johnson,TOR,1.81,ARI,True,1983,30,900
...,...,...,...,...,...,...,...,...,...
7404,2019,Marco Gonzales,SEA,3.21,SEA,False,1993,27,729
7405,2018,Marco Gonzales,SEA,2.48,SEA,False,1993,26,676
7406,2017,Marco Gonzales,SEA,0.01,STL,True,1993,25,625
7407,2015,Marco Gonzales,STL,-0.21,STL,False,1993,23,529


## 剩餘合約年相關變數、守備位置、平均年薪 (串FA_df、EX_df) 

### 剩餘合約年

In [12]:
# 串 FA_df
FA_df.rename(columns={"Year":"Season"},inplace=True)
WAR_df.rename(columns={"player":"Player"},inplace=True)
WAR_df.loc[:,"Season"] = WAR_df.loc[:,"Season"].astype(str)
FA_df.loc[:,"Season"] = FA_df.loc[:,"Season"].astype(str)
print(len(WAR_df))
WAR_df_m_contract = WAR_df.merge(FA_df[["Season","Player","Pos","YRS","AAV"]], how="left", on=["Player","Season"]).drop_duplicates(subset=["Player","Season","Team"])
print(len(WAR_df_m_contract))
WAR_df_m_contract.head()

7409
7409


,Season,Player,Team,WAR,preSeasonTeam,is_changeTeam,birth_year,Age,Age^2,Pos,YRS,AAV
0,2016,Kelly Johnson,ATL,-0.73,ATL,False,1983,34,1156,2B,1.0,"$2,000,000"
1,2015,Kelly Johnson,ATL,-0.04,NYY,True,1983,33,1089,2B,1.0,"$1,500,000"
2,2014,Kelly Johnson,NYY,-0.08,TB,True,1983,32,1024,2B,1.0,"$3,000,000"
3,2013,Kelly Johnson,TB,1.33,TOR,True,1983,31,961,2B,1.0,"$2,450,000"
4,2012,Kelly Johnson,TOR,1.81,ARI,True,1983,30,900,NaN,NaN,NaN


In [13]:
# 串 EX_df
EX_df.rename(columns={"year":"Season","Yrs":"YRS","Pos.":"Pos"},inplace=True)
EX_df["is_EX"] = True
EX_df.loc[:,"Season"] = EX_df.loc[:,"Season"].astype(str)

mask = WAR_df_m_contract["YRS"].isna()
print(len(WAR_df_m_contract.loc[mask,:]))
WAR_df_m_contract.loc[mask,:] = WAR_df_m_contract.loc[mask,WAR_df.columns].merge(
                                    EX_df[["Season","Player","Pos","YRS","AAV","is_EX"]],
                                        how="left", on=["Player","Season"]).set_index(WAR_df_m_contract.loc[mask,:].index)
print(len(WAR_df_m_contract.loc[WAR_df_m_contract["YRS"].isna(),:]))
WAR_df_m_contract.head()

6224
6123


,Season,Player,Team,WAR,preSeasonTeam,is_changeTeam,birth_year,Age,Age^2,Pos,YRS,AAV
0,2016,Kelly Johnson,ATL,-0.73,ATL,False,1983,34,1156,2B,1.0,"$2,000,000"
1,2015,Kelly Johnson,ATL,-0.04,NYY,True,1983,33,1089,2B,1.0,"$1,500,000"
2,2014,Kelly Johnson,NYY,-0.08,TB,True,1983,32,1024,2B,1.0,"$3,000,000"
3,2013,Kelly Johnson,TB,1.33,TOR,True,1983,31,961,2B,1.0,"$2,450,000"
4,2012,Kelly Johnson,TOR,1.81,ARI,True,1983,30,900,NaN,NaN,NaN


In [14]:
def YRSLeft_AAV_countter(df_sort_gp):
    '''
    計算複數年合約還剩下多少年、AAV複製到每個合約執行年中
    用法: 
    YRS_left_countter_all_dict = {}
    for df_sort_gp in WAR_df_m_contract.sort_values(by=["Season"]).groupby(["Player"], as_index=False):
        YRS_left_countter_all_dict.update(YRS_left_countter(df_sort_gp[1]))
        
    input: groupby 以排序的df
    output: dict(player:ans_lst)
    '''
    tmp =  df_sort_gp.loc[:,["Season","YRS","AAV"]]
    player = df_sort_gp.loc[:,"Player"].values[0]
    
    # 有複數年約的
    if tmp["YRS"].max()>1:
        for y in range(1,int(tmp["YRS"].max())+1):
            tmp.loc[:,f"YRS_s{y}"] =  tmp.loc[:,"YRS"].shift(y)

        # 逐行判斷
        YRS_recoder=0
        ans_lst=[]
        AAV_recoder=None
        AAV_lst=[]
        for row in tmp.iterrows():
            row = row[1]
            
            if pd.notna(row[f"YRS"]):
                YRS_recoder = row[f"YRS"]
                ans_lst.append(row[f"YRS"])
                
                AAV_recoder = row[f"AAV"]
                AAV_lst.append(row[f"AAV"])

            elif (YRS_recoder>1) & (any([pd.notna(row[f"YRS_s{y1}"]) for y1 in range(1,int(tmp["YRS"].max())+1)])):
                for y2 in range(1,int(tmp["YRS"].max())+1):
                    if (YRS_recoder==row[f"YRS_s{y2}"]):
                        ans_lst.append(YRS_recoder-y2)
                        AAV_lst.append(AAV_recoder)
                        break

            else:
                ans_lst.append(np.nan)
                AAV_lst.append(np.nan)
        return {player:ans_lst}, {player:AAV_lst}
    else: 
        return {player:tmp["YRS"].tolist()}, {player:tmp["AAV"].tolist()}

In [15]:
YRS_left_all_dict = {}
AAV_all_dict = {}
for df_sort_gp in WAR_df_m_contract.sort_values(by=["Season"]).groupby(["Player"], as_index=False):
    ans = YRSLeft_AAV_countter(df_sort_gp[1])
    YRS_left_all_dict.update(ans[0])
    AAV_all_dict.update(ans[1])

In [16]:
WAR_df_m_contract.sort_values(by=["Season"], inplace=True)
for player in YRS_left_all_dict.keys():
    try:
        WAR_df_m_contract.loc[WAR_df_m_contract["Player"]==player, "YRS_left"] = YRS_left_all_dict[player]
        WAR_df_m_contract.loc[WAR_df_m_contract["Player"]==player, "AAV_dollar"] = AAV_all_dict[player]
    except:
        print(player)

In [17]:
# 簽約年虛擬變數
# 是否為1年約
WAR_df_m_contract.loc[:,"is_1year_contract"] = WAR_df_m_contract.loc[:,"YRS"].apply(lambda y: y==1)

#是否為複數年合約第一年
WAR_df_m_contract.loc[:,"YRS_shift1"] = WAR_df_m_contract.groupby(["Player"])["YRS"].shift()
WAR_df_m_contract.loc[:,"is_FIRST_year_BigContract"] = False
mask = WAR_df_m_contract["YRS_shift1"].notna()
WAR_df_m_contract.loc[mask,"is_FIRST_year_BigContract"] = ((WAR_df_m_contract.loc[mask,"YRS_shift1"]-1)==WAR_df_m_contract.loc[mask,"YRS_left"])

#是否為複數年剩下2、3、4(以上)年
WAR_df_m_contract.loc[:,"is_left_2years"] = WAR_df_m_contract.loc[:,"YRS_left"]==2
WAR_df_m_contract.loc[:,"is_left_3years"] = WAR_df_m_contract.loc[:,"YRS_left"]==3
WAR_df_m_contract.loc[:,"is_left_ge4years"] = WAR_df_m_contract.loc[:,"YRS_left"]>=4

# 是否為最後出賽年
player_max_season_df = WAR_df_m_contract[["Player","Season"]].groupby(["Player"], as_index=False).max()
player_max_season_df["is_END_year"] = True
WAR_df_m_contract = WAR_df_m_contract.merge(player_max_season_df[~player_max_season_df["Season"].isin(["2021"])], how="left", on=["Player","Season"])
WAR_df_m_contract["is_END_year"].fillna(False, inplace=True)

## 平均年薪(AAV)格式整理

In [18]:
# $符號轉數字
mask = WAR_df_m_contract.AAV_dollar.notna()
WAR_df_m_contract.loc[mask,"AAV_dollar"] =WAR_df_m_contract.loc[mask,"AAV_dollar"].apply(
                                                                    lambda money: money.split("$")[-1].replace(",","") 
                                                                                if pd.notna(money) else money)

WAR_df_m_contract.loc[mask,"AAV_dollar"] = pd.to_numeric(WAR_df_m_contract.loc[mask,"AAV_dollar"], errors='coerce')
WAR_df_m_contract.loc[:,"AAV_dollar"] = WAR_df_m_contract.loc[:,"AAV_dollar"].fillna(0).astype(int)

In [19]:
# WAR_df_m_contract[WAR_df_m_contract["Player"].isin(["Mookie Betts"])]

## 為 Player 設定一組數字ID>> Label encoding

In [20]:
WAR_df_m_contract.loc[:,"Player"] = WAR_df_m_contract["Player"].astype("category")
WAR_df_m_contract.loc[:,"Player_ID"] = WAR_df_m_contract["Player"].cat.codes

## 格式轉換for輸出
WAR_df_m_contract.loc[:,"Season"] = WAR_df_m_contract.loc[:,"Season"].astype(np.int64)

## 所屬球隊、守備位置 做 dummy

In [21]:
print(WAR_df_m_contract["Team"].unique())
print(WAR_df_m_contract["Team"].nunique()) # GOOD!
# # dummy drop_first=True
WAR_df_m_contract = pd.concat([WAR_df_m_contract, pd.get_dummies(WAR_df_m_contract.Team, prefix='Team', drop_first=True)], axis=1)
WAR_df_m_contract.columns

['PHI' 'BOS' 'NYM' 'CIN' 'TOR' 'LAD' 'WSH' 'PIT' 'SEA' 'BAL' 'TB' 'NYY'
 'OAK' 'COL' 'MIN' 'CHW' 'MIA' 'MIL' 'STL' 'SD' 'LAA' 'KC' 'ATL' 'DET'
 'ARI' 'CLE' 'HOU' 'TEX' 'SF' 'CHC']
30


Index(['Season', 'Player', 'Team', 'WAR', 'preSeasonTeam', 'is_changeTeam',
       'birth_year', 'Age', 'Age^2', 'Pos', 'YRS', 'AAV', 'YRS_left',
       'AAV_dollar', 'is_1year_contract', 'YRS_shift1',
       'is_FIRST_year_BigContract', 'is_left_2years', 'is_left_3years',
       'is_left_ge4years', 'is_END_year', 'Player_ID', 'Team_ATL', 'Team_BAL',
       'Team_BOS', 'Team_CHC', 'Team_CHW', 'Team_CIN', 'Team_CLE', 'Team_COL',
       'Team_DET', 'Team_HOU', 'Team_KC', 'Team_LAA', 'Team_LAD', 'Team_MIA',
       'Team_MIL', 'Team_MIN', 'Team_NYM', 'Team_NYY', 'Team_OAK', 'Team_PHI',
       'Team_PIT', 'Team_SD', 'Team_SEA', 'Team_SF', 'Team_STL', 'Team_TB',
       'Team_TEX', 'Team_TOR', 'Team_WSH'],
      dtype='object')

In [22]:
# Pos 回填
Player_Pos_dict = {row[1]["Player"]:row[1]["Pos"] for row in WAR_df_m_contract.loc[WAR_df_m_contract["Pos"].notna(),["Player","Pos"]].drop_duplicates().iterrows()}
WAR_df_m_contract.loc[:,"Pos"] = WAR_df_m_contract.loc[:,"Player"].map(Player_Pos_dict)

# RP/CL >> RP
WAR_df_m_contract.loc[WAR_df_m_contract["Pos"].isin(["RP/CL"]),"Pos"] = "RP"

# 刪除沒Pos的資料
print(len(WAR_df_m_contract))
WAR_df_m_contract = WAR_df_m_contract.dropna(subset=["Pos"])
print(len(WAR_df_m_contract))

7409
6901


### 拆成 pitch、bat 分別做Pos dummy

In [23]:
# pitch or bat
WAR_df_m_contract.loc[WAR_df_m_contract["Pos"].isin(["RP","SP"]),"type"] = "pitch"
WAR_df_m_contract.loc[WAR_df_m_contract["Pos"].isin(["C","2B","1B","3B","LF","RF","CF","SS","DH","OF"]),"type"] = "bat"

# 拆分資料
pitch_data = WAR_df_m_contract[WAR_df_m_contract["type"].isin(["pitch"])].reset_index()
bat_data = WAR_df_m_contract[WAR_df_m_contract["type"].isin(["bat"])].reset_index()

# dummy drop_first=True
pitch_data = pd.concat([pitch_data, pd.get_dummies(pitch_data.Pos, prefix='Pos', drop_first=True)], axis=1)
print(pitch_data.columns)
bat_data = pd.concat([bat_data, pd.get_dummies(bat_data.Pos, prefix='Pos', drop_first=True)], axis=1)
print(bat_data.columns)

Index(['index', 'Season', 'Player', 'Team', 'WAR', 'preSeasonTeam',
       'is_changeTeam', 'birth_year', 'Age', 'Age^2', 'Pos', 'YRS', 'AAV',
       'YRS_left', 'AAV_dollar', 'is_1year_contract', 'YRS_shift1',
       'is_FIRST_year_BigContract', 'is_left_2years', 'is_left_3years',
       'is_left_ge4years', 'is_END_year', 'Player_ID', 'Team_ATL', 'Team_BAL',
       'Team_BOS', 'Team_CHC', 'Team_CHW', 'Team_CIN', 'Team_CLE', 'Team_COL',
       'Team_DET', 'Team_HOU', 'Team_KC', 'Team_LAA', 'Team_LAD', 'Team_MIA',
       'Team_MIL', 'Team_MIN', 'Team_NYM', 'Team_NYY', 'Team_OAK', 'Team_PHI',
       'Team_PIT', 'Team_SD', 'Team_SEA', 'Team_SF', 'Team_STL', 'Team_TB',
       'Team_TEX', 'Team_TOR', 'Team_WSH', 'type', 'Pos_SP'],
      dtype='object')
Index(['index', 'Season', 'Player', 'Team', 'WAR', 'preSeasonTeam',
       'is_changeTeam', 'birth_year', 'Age', 'Age^2', 'Pos', 'YRS', 'AAV',
       'YRS_left', 'AAV_dollar', 'is_1year_contract', 'YRS_shift1',
       'is_FIRST_year_BigContra

In [24]:
bat_data[bat_data["Player"].isin(["Mookie Betts"])]

,index,Season,Player,Team,WAR,preSeasonTeam,is_changeTeam,birth_year,Age,Age^2,...,type,Pos_2B,Pos_3B,Pos_C,Pos_CF,Pos_DH,Pos_LF,Pos_OF,Pos_RF,Pos_SS
1827,3893,2014,Mookie Betts,BOS,2.28,NaN,False,1994,21,441,...,bat,0,0,0,0,0,0,0,1,0
2259,4798,2015,Mookie Betts,BOS,5.86,BOS,False,1994,22,484,...,bat,0,0,0,0,0,0,0,1,0
2476,5238,2016,Mookie Betts,BOS,9.72,BOS,False,1994,23,529,...,bat,0,0,0,0,0,0,0,1,0
2697,5704,2017,Mookie Betts,BOS,6.44,BOS,False,1994,24,576,...,bat,0,0,0,0,0,0,0,1,0
2851,6045,2018,Mookie Betts,BOS,10.86,BOS,False,1994,25,625,...,bat,0,0,0,0,0,0,0,1,0
3115,6610,2019,Mookie Betts,BOS,6.82,BOS,False,1994,26,676,...,bat,0,0,0,0,0,0,0,1,0
3341,7078,2020,Mookie Betts,LAD,3.37,BOS,True,1994,27,729,...,bat,0,0,0,0,0,0,0,1,0
3463,7345,2021,Mookie Betts,LAD,3.79,LAD,False,1994,28,784,...,bat,0,0,0,0,0,0,0,1,0


# 輸出

In [25]:
col_output_bat = [
#  'index',
 'Season',
 'Player',
 'Team',
 'WAR',
#  'preSeasonTeam',
 'is_changeTeam',
#  'birth_year',
 'Age',
 'Age^2',
 'Pos',
 'YRS',
#  'AAV',
 'YRS_left',
 'AAV_dollar',
 'is_1year_contract',
#  'YRS_shift1',
 'is_FIRST_year_BigContract',
 'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 'Player_ID',
 'Team_ATL',
 'Team_BAL',
 'Team_BOS',
 'Team_CHC',
 'Team_CHW',
 'Team_CIN',
 'Team_CLE',
 'Team_COL',
 'Team_DET',
 'Team_HOU',
 'Team_KC',
 'Team_LAA',
 'Team_LAD',
 'Team_MIA',
 'Team_MIL',
 'Team_MIN',
 'Team_NYM',
 'Team_NYY',
 'Team_OAK',
 'Team_PHI',
 'Team_PIT',
 'Team_SD',
 'Team_SEA',
 'Team_SF',
 'Team_STL',
 'Team_TB',
 'Team_TEX',
 'Team_TOR',
 'Team_WSH',
 'type',
 'Pos_2B',
 'Pos_3B',
 'Pos_C',
 'Pos_CF',
 'Pos_DH',
 'Pos_LF',
 'Pos_OF',
 'Pos_RF',
 'Pos_SS']


formtoXlsx(bat_data[col_output_bat], "Spotrac_bat_data")
bat_data[col_output_bat].to_stata('Spotrac_bat_data.dta')

 ==輸出備份xlsx== ::: Spotrac_bat_data_20210828_19.48.49.xlsx


G:\我的雲端硬碟\python\polib\CsvEngn.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  form_data["_idx"] = form_data.index
C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\pandas\io\stata.py:2397: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Age^2   ->   Age_2

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  warnings.warn(ws, InvalidColumnName)


In [26]:
col_output_pitch = [
#  'index',
 'Season',
 'Player',
 'Team',
 'WAR',
#  'preSeasonTeam',
 'is_changeTeam',
#  'birth_year',
 'Age',
 'Age^2',
 'Pos',
 'YRS',
#  'AAV',
 'YRS_left',
 'AAV_dollar',
 'is_1year_contract',
#  'YRS_shift1',
 'is_FIRST_year_BigContract',
 'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 'Player_ID',
 'Team_ATL',
 'Team_BAL',
 'Team_BOS',
 'Team_CHC',
 'Team_CHW',
 'Team_CIN',
 'Team_CLE',
 'Team_COL',
 'Team_DET',
 'Team_HOU',
 'Team_KC',
 'Team_LAA',
 'Team_LAD',
 'Team_MIA',
 'Team_MIL',
 'Team_MIN',
 'Team_NYM',
 'Team_NYY',
 'Team_OAK',
 'Team_PHI',
 'Team_PIT',
 'Team_SD',
 'Team_SEA',
 'Team_SF',
 'Team_STL',
 'Team_TB',
 'Team_TEX',
 'Team_TOR',
 'Team_WSH',
 'type',
 'Pos_SP']

formtoXlsx(pitch_data[col_output_pitch], "Spotrac_pitch_data")
pitch_data[col_output_pitch].to_stata('Spotrac_pitch_data.dta')

G:\我的雲端硬碟\python\polib\CsvEngn.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  form_data["_idx"] = form_data.index


 ==輸出備份xlsx== ::: Spotrac_pitch_data_20210828_19.48.58.xlsx


C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\pandas\io\stata.py:2397: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Age^2   ->   Age_2

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  warnings.warn(ws, InvalidColumnName)
